<a href="https://colab.research.google.com/github/OsVox/YouTube-predictions/blob/main/YouTube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing modules

In [1]:
!pip install tensorflow_text
!pip install pyyaml h5py

     |████████████████████████████████| 4.9 MB 8.6 MB/s 


In [2]:
import os
import gc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
import torch
import tensorflow_hub as hub
import tensorflow_text as text

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
# from transformers import BertTokenizer, BertModel, AutoTokenizer

In [ ]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)

# Reading data

In [ ]:
from google.colab import drive
mount_directory = "/content/drive"

In [ ]:
test_df = pd.read_parquet('/content/drive/MyDrive/colab/test')
train_df = pd.read_parquet('/content/drive/MyDrive/colab/train')

train = train_df.copy()
test = test_df.copy()

In [ ]:
train

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,description,id,duration_seconds,has_thumbnail,target,has_description,target_prediction
0,3C66w5Z0ixs,i asked her to be my girlfriend...,2020-08-11 19:20:14+00:00,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,3C66w5Z0ixs_2020-08-12,547.0,True,0.103596,True,0.061548
1,M9Pmf9AB4Mo,apex legends | stories from the outlands – “th...,2020-08-11 17:00:10+00:00,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,"While running her own modding shop, Ramya Pare...",M9Pmf9AB4Mo_2020-08-12,662.0,True,0.061611,True,0.057524
2,J78aPJ3VyNs,i left youtube for a month and this is what ha...,2020-08-11 16:34:06+00:00,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,I left youtube for a month and this is what ha...,J78aPJ3VyNs_2020-08-12,528.0,True,0.173523,True,0.124688
3,kXLn3HkpjaA,xxl 2020 freshman class revealed - official an...,2020-08-11 16:38:55+00:00,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...,kXLn3HkpjaA_2020-08-12,203.0,True,0.046804,True,0.039038
4,VIUo6yapDbc,ultimate diy home movie theater for the labran...,2020-08-11 15:10:05+00:00,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,Transforming The LaBrant Family's empty white ...,VIUo6yapDbc_2020-08-12,836.0,True,0.040753,True,0.054775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91826,H1tQhK0n5Qk,thanks for 5 million,2021-11-23 19:13:46+00:00,UCSUf5_EPEfl4zlBKZHkZdmw,Danny Gonzalez,23,2021-11-30,danny gonzalez|funny|commentary|comedy|react|r...,536056,109352,550,4374,https://i.ytimg.com/vi/H1tQhK0n5Qk/default.jpg,False,#shorts,H1tQhK0n5Qk_2021-11-30,59.0,True,0.203994,True,0.090401
91827,3CuO39VL3rk,mrs. rogers done f’ed up😅 #shorts #story #come...,2021-11-24 23:48:37+00:00,UCqseWqrl7r-hNoWme7hIfTw,Rebecca Rogers (Mrs. Rogers),23,2021-11-30,[None],863236,116742,1161,583,https://i.ytimg.com/vi/3CuO39VL3rk/default.jpg,False,No description,3CuO39VL3rk_2021-11-30,58.0,True,0.135238,False,0.057188
91828,qfDVsuwg0TU,my new kitchen reveal!!!!! exciting,2021-11-23 22:50:19+00:00,UCnB4LOZe6Dc5JUH9EoaHhKQ,Bri & Fern,1,2021-11-30,10-19-21,194119,14569,180,5276,https://i.ytimg.com/vi/qfDVsuwg0TU/default.jpg,False,Contracter email: Los3topetes@gmail.comFOLLOW ...,qfDVsuwg0TU_2021-11-30,824.0,True,0.075052,True,0.057801
91829,HHbF8HaC4w8,using reverse psychology to get anything you want,2021-11-21 21:15:16+00:00,UCQNDoTI2g7_daH6zL_9rvWg,Amyy Woahh,23,2021-11-30,[None],24584472,1457618,61893,3610,https://i.ytimg.com/vi/HHbF8HaC4w8/default.jpg,False,No description,HHbF8HaC4w8_2021-11-30,56.0,True,0.059290,False,0.076928


# Basic secription of data

In [ ]:
# There are a couple of null values, let's deal with them.
# Before we do something to our df, let's unite them
# in order to change them simultaneously

In [ ]:
dfs = [train, test]

In [ ]:
a = train[train.publishedAt.dt.year == 2021].groupby('categoryId').duration_seconds.median()

In [ ]:
for df in dfs:
    for category in a.index:
        df.loc[(df.duration_seconds.isna()) & (df.categoryId == category), 'duration_seconds'] = a[category]

In [ ]:
train.isnull().sum()

In [ ]:
# We should create a new column, which shows whether discription is NaN or not

In [ ]:
for df in dfs:
    df['has_description'] = ~df['description'].isnull()

In [ ]:
for df in dfs:
    df.description.fillna('No description', inplace=True)

In [ ]:
for df in dfs:
    df.trending_date = pd.to_datetime(df.trending_date)

In [ ]:
for df in dfs:
    df.drop(df[df.ratings_disabled == True].index, inplace=True)

In [ ]:
train.title = train.title.str.lower()
titles = list(train.title)

In [ ]:
train[train.ratings_disabled == True].index

In [ ]:
train.reset_index(inplace=True)

# Using BERT to deal with titles, descriptions and tags

## Making embeddings

In [ ]:
MAKE_EMB = False
if MAKE_EMB:
    for i in range(0, len(titles) // 32 + 1):
        np.save('/content/drive/MyDrive/embeddings/em_{}'.format(i), \
                bert_model(bert_preprocess_model(titles[32*i : 32*(i + 1)])) \
                ['pooled_output'].numpy())
        gc.collect()

## Creating model

In [ ]:
def yield_batches():
    a = np.load('/content/drive/MyDrive/embeddings/em_{}.npy'.format(0))
    b = np.array(train.target[0:32])

    for i in range(1, 2870):
        ar = np.load('/content/drive/MyDrive/embeddings/em_{}.npy'.format(i))
        a = np.concatenate([a, ar])
        b = np.concatenate([b, np.array(train.target[32*i:32*(i+1)])])


    return a, b

In [ ]:
model = Sequential()
model.add(Dense(768, input_shape=(768,), activation='elu'))
model.add(Dense(512, activation='elu'))
model.add(Dense(512, activation='elu'))
model.add(Dense(256, activation='elu'))
model.add(Dense(128, activation='elu'))
keras.layers.Dropout(rate=0.2) 
model.add(Dense(128, activation='elu'))
model.add(Dense(64, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1, activation='elu'))

In [ ]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

model.compile(
    optimizer=adam_optimizer,
    loss=tf.keras.losses.MeanAbsoluteError(
    reduction="auto", name="mean_absolute_error"),
)

In [ ]:
USE_GOOGLE_DRIVE = True

def mount_google_drive():
    drive.mount(mount_directory)
    drive_root = mount_directory + "/" + list(filter(lambda x: x[0] != '.', os.listdir(mount_directory)))[0] + "/colab"
    return drive_root

CHECKPOINT_ROOT = ""
if USE_GOOGLE_DRIVE:
    import datetime
    CHECKPOINT_ROOT = mount_google_drive() + "/"

def get_checkpoint_path(epoch=None):
    if epoch is None:
        return os.path.abspath(CHECKPOINT_ROOT + "weights")
    else:
        return os.path.abspath(CHECKPOINT_ROOT + "weights_{}".format(epoch))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
checkpoint_path = "/content/drive/MyDrive/colab/Checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
if MAKE_EMB:
    train_embeddings, train_y = yield_batches()
    np.save('/content/drive/MyDrive/embeddings/train_embeddings.npy', train_embeddings)
    np.save('/content/drive/MyDrive/embeddings/train_y.npy', train_y)

In [ ]:
train_embeddings = np.load('/content/drive/MyDrive/train_embeddings.npy')
train_y = np.load('/content/drive/MyDrive/train_y.npy')

In [ ]:
model.load_weights("/content/drive/MyDrive/colab/Checkpoints/cp.ckpt")

In [ ]:
TRAIN_MODEL = False

if TRAIN_MODEL:
    training_history = model.fit(
        train_embeddings,
        train_y,
        epochs=100,
        callbacks=[cp_callback],
        batch_size=64, 
        validation_split=0.1, 
        shuffle=True
    )

## making predictions

In [ ]:
def predict(text):
    pre = bert_preprocess_model(text)
    emb = bert_model(pre)['pooled_output']
    return model.predict(emb)

In [ ]:
train.to_parquet('/content/drive/MyDrive/colab/train_1')

In [ ]:
def f(lst):
    lst += ['[NONE]'] * max(0, 10 - len(lst))
    lst = np.random.choice(lst, size = 10, replace=False)
    return lst

In [ ]:
def stretch(array):
    array = np.array([[element] * 10 for element in array])
    return array.reshape(-1)

In [ ]:
for i in range(0, 90):    
    tags_embeddings = np.array(list(map(lambda x: bert_model(bert_preprocess_model(x))['pooled_output'], np.array(tags[1024*i: 1024*(i+1)]))))
    np.save('/content/drive/MyDrive/embeddings/em_{}'.format(i), tags_embeddings)
    print(tags_embeddings.shape)

(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)
(1024, 10, 768)


KeyboardInterrupt: ignored

In [ ]:
def yield_batches_2():
    a = np.load('/content/drive/MyDrive/embeddings/em_{}.npy'.format(0))
    b = np.array(train.target[0:1024])

    for i in range(1, 30):
        ar = np.load('/content/drive/MyDrive/embeddings/em_{}.npy'.format(i))
        a = np.concatenate([a, ar])
        b = np.concatenate([b, np.array(train.target[1024*i:1024*(i+1)])])


    return a, b

In [ ]:
train_emb, train_y = yield_batches_2()

In [ ]:
np.save('/content/drive/MyDrive/embeddings/train_embeddings', train_emb)

In [ ]:
train_emb = train_emb.reshape(307200, 768)

In [ ]:
train_y = stretch(train_y)

In [ ]:
model = Sequential()
model.add(Dense(768, input_shape=(768,), activation='elu'))
model.add(Dense(512, activation='elu'))
model.add(Dense(512, activation='elu'))
model.add(Dense(256, activation='elu'))
model.add(Dense(128, activation='elu'))
keras.layers.Dropout(rate=0.2) 
model.add(Dense(128, activation='elu'))
model.add(Dense(64, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1, activation='elu'))

adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(
    optimizer=adam_optimizer,
    loss=tf.keras.losses.MeanAbsoluteError(
    reduction="auto", name="mean_absolute_error"),
)

In [ ]:
checkpoint_path = "/content/drive/MyDrive/colab/Checkpoints/cp2.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
TRAIN_MODEL = True

if TRAIN_MODEL:
    training_history = model.fit(
        train_emb,
        train_y,
        epochs=100,
        callbacks=[cp_callback],
        batch_size=64, 
        validation_split=0.1, 
        shuffle=True
    )

Epoch 1/100
4313/4320 [============================>.] - ETA: 0s - loss: 0.0335
Epoch 00001: saving model to /content/drive/MyDrive/colab/Checkpoints/cp2.ckpt
4320/4320 [==============================] - 28s 6ms/step - loss: 0.0335 - val_loss: 0.0274
Epoch 2/100
4315/4320 [============================>.] - ETA: 0s - loss: 0.0297
Epoch 00002: saving model to /content/drive/MyDrive/colab/Checkpoints/cp2.ckpt
4320/4320 [==============================] - 22s 5ms/step - loss: 0.0297 - val_loss: 0.0293
Epoch 3/100
4313/4320 [============================>.] - ETA: 0s - loss: 0.0291
Epoch 00003: saving model to /content/drive/MyDrive/colab/Checkpoints/cp2.ckpt
4320/4320 [==============================] - 22s 5ms/step - loss: 0.0291 - val_loss: 0.0326
Epoch 4/100
4320/4320 [==============================] - ETA: 0s - loss: 0.0287
Epoch 00004: saving model to /content/drive/MyDrive/colab/Checkpoints/cp2.ckpt
4320/4320 [==============================] - 25s 6ms/step - loss: 0.0287 - val_loss: 0.0

In [ ]:
train['error'] = abs(train.target - train.target_prediction)

In [ ]:
train.target.std()

0.04027458325922316

In [ ]:
train.target_prediction.std()

0.029148858040571213

In [ ]:
train.error.mean()

0.015533054241359643

In [ ]:
train.sort_values(by='error', ascending=False)

In [ ]:
model.load_weights('/content/drive/MyDrive/colab/Checkpoints/cp2.ckpt')

In [ ]:
train.tags[:10]

0    [bmw i8, browadis, skits, mama rug and papa ru...
1    [Apex Legends game, multiplayer shooter, Apex ...
2    [funny memes, funny meme, jacksepticeye memes,...
3    [2020 xxl freshman, 2020 freshman, hip-hop, Li...
4    [beautiful, Norton, Home title scams, luxuriou...
5    [[NONE], Professor injury, professorlive injur...
6    [[NONE], [None], [NONE], [NONE], [NONE], [NONE...
7    [[NONE], education, [NONE], [NONE], cgpgrey, h...
8    [dad, makeup, dream, mom, parents, the baddest...
9    [Alta Consigna, Vengo De Nada, Herencia de Pat...
Name: tags, dtype: object

In [ ]:
train

In [ ]:
train.tags = train.tags.apply(lambda x: x.split('|')) 
tags = list(train.tags)
tags = list(filter(lambda x: len(x) > 8, tags))
len(tags)

63465

In [ ]:
k_list = []
i = 0

for video in tags[:100]:
    emb = bert_model(bert_preprocess_model(video))
    kmeans = KMeans(8).fit(emb['pooled_output'])
    k_list.append(kmeans.labels_)
    i += 1
    if i % 20 == 0:
        np.save('/content/drive/MyDrive/colab/k_means_{}'.format(i), k_list)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with differe